In [73]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'merge_str_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/merge_str_1/outputs/results/run']


In [74]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [75]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration.columns)
print(overall_duration.index)
print(overall_duration)
throughput = overall_duration
'''
for column in throughput.columns:
    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])
    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0
print(throughput)
overall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() 
overall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() 
'''

Index(['btree1024', 'btree2048', 'btree256', 'pgm1024', 'pgm2048', 'pgm256',
       'standard_merge'],
      dtype='object', name='spec.algo_name')
Index([1, 5, 10, 50, 100], dtype='int64', name='spec.common_key')
spec.algo_name      btree1024     btree2048      btree256       pgm1024  \
spec.common_key                                                           
1                         NaN           NaN           NaN           NaN   
5                1.785659e+10  1.515691e+10  1.965494e+10  6.398384e+09   
10               1.110211e+10  9.602647e+09  1.210193e+10  4.768641e+09   
50               4.549987e+09  4.233646e+09  4.783735e+09  3.199191e+09   
100              3.657186e+09  3.500125e+09  3.778569e+09  2.981610e+09   

spec.algo_name        pgm2048        pgm256  standard_merge  
spec.common_key                                              
1                         NaN  2.949066e+09             NaN  
5                6.333842e+09  6.523274e+09    5.241529e+09  
10          

'\nfor column in throughput.columns:\n    throughput[column] = (200_000_000/throughput.index) * (1000_000_000.0 / throughput[column])\n    throughput[column +"_sj-rel"] = (throughput[column] - throughput["sj"] / throughput["sj"]) * 100.0\nprint(throughput)\noverall_duration[["btree2048", "btree1024", "btree256", "sj"]].plot() \noverall_duration[["pgm128", "pgm512", "pgm1024", "sj"]].plot() \n'

In [76]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
inner_index_total_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_total_bytes_fetched', aggfunc='median')
inner_index_fetch_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch_size', aggfunc='median')
print(inner_index_fetch)
print(inner_index_total_fetch)
print(inner_index_fetch_size)

spec.algo_name   btree1024  btree2048  btree256   pgm1024   pgm2048    pgm256  \
spec.common_key                                                                 
1                      NaN        NaN       NaN       NaN       NaN  781250.0   
5                 781250.0   781250.0  781250.0  781250.0  781250.0  781250.0   
10                781250.0   781250.0  781250.0  781250.0  781250.0  781250.0   
50                781250.0   781250.0  781250.0  781250.0  781250.0  781250.0   
100               781250.0   781250.0  781250.0  781250.0  781250.0  781250.0   

spec.algo_name   standard_merge  
spec.common_key                  
1                           NaN  
5                      781250.0  
10                     781250.0  
50                     781250.0  
100                    781250.0  
spec.algo_name      btree1024     btree2048      btree256       pgm1024  \
spec.common_key                                                           
1                         NaN           NaN 

In [77]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 100])
print(result_checksum.loc[result_checksum['spec.common_key'] == 100]['result.checksum'].unique())

common_key: 1 checksum: [0] OK
common_key: 5 checksum: [0] OK
common_key: 10 checksum: [0] OK
common_key: 50 checksum: [0] OK
common_key: 100 checksum: [0] OK
    spec.common_key  spec.algo_name  result.checksum
47              100       btree1024                0
60              100       btree1024                0
76              100       btree1024                0
3               100       btree2048                0
29              100       btree2048                0
68              100       btree2048                0
72              100        btree256                0
77              100        btree256                0
78              100        btree256                0
14              100         pgm1024                0
45              100         pgm1024                0
50              100         pgm1024                0
33              100         pgm2048                0
58              100         pgm2048                0
75              100         pgm2048           